In [ ]:
import numpy as np
from itertools import permutations

from libs import geo # Importing the intercept module
from libs import constants as cst  # Importing constants for speed conversion
from libs.transform import Transform  # Importing the Transform class for coordinate transformations
from libs.trajectory import Trajectory  # Importing the Trajectory class for trajectory calculations
from libs import trajectory as traj
from libs import intercept  # Importing the intercept module for trajectory calculations
from libs import tacview  # Importing the tacview module for writing ACMI files


In [ ]:
#Setup
coord_f= Transform(lat=34.9054961, lon=-117.8838, alt=10000*cst.FT_TO_METER, roll=0,pitch=0,yaw=90)
TAS_f = 500 * cst.KT_TO_MS  # Fighter's True Air Speed in m/s
load_factor_f = 3.0  # Load factor for the fighter

coord_tgt1=geo.aer2geodetic(5,0,20*cst.NM_TO_M, coord_f,level=True)
coord_tgt1.yaw=270
coord_tgt2=geo.aer2geodetic(-4,0,18*cst.NM_TO_M, coord_f,level=True)
coord_tgt2.yaw=265
coord_tgt3=geo.aer2geodetic(15,0,35*cst.NM_TO_M, coord_f,level=True)
coord_tgt3.yaw=280

TAS_tgt = 200 * cst.KT_TO_MS  # Same airspeed for all targets

list_tgt = [coord_tgt1, coord_tgt2, coord_tgt3]
list_tgt_name= ['Target 1', 'Target 2', 'Target 3']

#get all the possibilities of target coordinates
tgt_permutations = np.array(list(permutations(list_tgt, len(list_tgt))))
tgt_name_permutations = np.array(list(permutations(list_tgt_name, len(list_tgt_name))))


In [ ]:
trajectories_f = []  # List to store all trajectories for plotting
max_time = 0  # Variable to keep track of the maximum time across all trajectories
min_time = 10e6  # Variable to keep track of the minimum time across all trajectories
best_traj = None  # Variable to store the best trajectory found

for i, perm in enumerate(tgt_permutations):
    params = {'Name': 'L-39', 'Pilot': 'Fighter' + str(i+1), 'Type': 'Air+FixedWing', 'Color': 'Cyan'}  # Parameters for the target trajectory
    ID= str(i+1)  # Unique ID for the trajectory
    traj_f1, _,_ = intercept.intercept(coord_f, perm[0], TAS_f,TAS_tgt, load_factor_f, plots=False, debug=False)

    #Get the start point for the second intercept
    time_inter1= traj_f1.times[-1]  # Get the time of the first intercept
    coord_tgt1=geo.straight_line(perm[1], TAS_tgt, time_inter1, level=True)  # Get the target coordinate at the intercept time
    traj_f2, _,_ = intercept.intercept(traj_f1.coords[-1], coord_tgt1, TAS_f,TAS_tgt, load_factor_f, plots=False, debug=False)
    traj_f2.time_offset(time_inter1)  # Offset the second trajectory to start after the first


    time_inter2 = traj_f2.times[-1]  # Get the time of the second intercept

    coord_tgt2=geo.straight_line(perm[2], TAS_tgt, time_inter2, level=True)  # Get the target coordinate at the second intercept time
    traj_f3, _,_ = intercept.intercept(traj_f2.coords[-1], coord_tgt2, TAS_f,TAS_tgt, load_factor_f, plots=False, debug=False)
    traj_f3.time_offset(time_inter2)  # Offset the third trajectory to start after the second
    # print(f"Intercepting target {perm[2]} at time {traj_f3.times[-1]} seconds.")
    traj_f=traj.merge_trajectories([traj_f1, traj_f2, traj_f3])  # Merge the trajectories of the three fighters
    # print("length of merged trajectory:", len(traj_f.times))
    # print(traj_f.times)
    if traj_f.times[-1] > max_time:  # Update max_time if the current trajectory's end time is greater
        max_time = traj_f.times[-1]
    if traj_f.times[-1] < min_time:  # Update min_time if the current trajectory's start time is less
        min_time = traj_f.times[-1]
        best_traj = i
    traj_f.params=params
    traj_f.ID = ID  # Assign the unique ID to the trajectory parameters
    trajectories_f.append(traj_f)  # Append the merged trajectory to the list
    trajectories_f[i].params['Color'] = 'Cyan'  # Change the color for the best trajectory
    # print(f"Trajectory {i+1} with ID {ID} has end time {traj_f.times[-1]} seconds.")

for traj_f in trajectories_f:
    if traj_f.times[-1] == min_time:
        traj_f.params['Color'] = 'Blue'

#Generate the trajectories for the targets
trajectories_tgt = []  # List to store all target trajectories
times_tgt=np.arange(0, max_time + 1, 1)  # Create a time array from 0 to max_time with a step of 1 second
print(f"Min time for target trajectories: {min_time:.2f} seconds going {tgt_name_permutations[i]}")
# print("Target names in permutation at index", i, ":", [tgt.params['Name'] for tgt in tgt_permutations[i]])
for i, tgt in enumerate(list_tgt):
    coords_tgt=[]
    params = {'Name': 'Kh-101','Pilot':'Target '+str(i+1), 'Type': 'Air+Missile', 'Color': 'Red'}  # Parameters for the target trajectory
    ID=str(101+i)  # Assign a unique ID to the target trajectory
    for j in range(len(times_tgt)):
        coord_tgt = geo.straight_line(tgt, TAS_tgt, times_tgt[j], level=True)
        coords_tgt.append(coord_tgt)
    coords_tgt = np.array(coords_tgt)  # Convert the list of coordinates to a numpy array
    traj_tgt = Trajectory(coords_tgt, times_tgt,params)  # Create a trajectory for the target
    traj_tgt.ID = ID
    traj_tgt.params=params
    trajectories_tgt.append(traj_tgt)  # Append the target trajectory to the list
    # print(f"Target trajectory {i+1} with ID {ID} has end time {traj_tgt.times[-1]} seconds.")
    all_trajectories = trajectories_f + trajectories_tgt

tacview.traj2tacview('intercept_swarm', all_trajectories)  # Write the fighter trajectories to a Tacview file

#Plot geodetic coordinate for best trajectory
